## Импорт библиотек

In [1]:
!pip install catboost


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 5.3 MB/s eta 0:00:00


In [2]:
%load_ext autoreload
%autoreload 2

import gc

import os
import sys
import pandas as pd
import numpy as np
import tqdm
import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.inspection import permutation_importance
from sklearn.metrics import roc_auc_score

import catboost as cb
import lightgbm as lgb

pd.set_option('display.max_columns', None)

os.environ["CUDA_VISIBLE_DEVICES"] = '0'


## Config

In [3]:
TRAIN_TRANSACTIONS_PATH = '/content/drive/My Drive/Университет/Credit_Scoring/data/train/'
TEST_TRANSACTIONS_PATH = '/content/drive/My Drive/Университет/Credit_Scoring/data/test/'
TRAIN_TARGET_PATH = '/content/drive/My Drive/Университет/Credit_Scoring/data/train_target.csv'
TEST_TARGET_PATH = '/content/drive/My Drive/Университет/Credit_Scoring/data/test_target_contest.csv'
FEATURE_EXTRACTION_PATH_TRAIN = '/content/drive/My Drive/Университет/Credit_Scoring/data/feature_extraction/train/'
FEATURE_EXTRACTION_PATH_TEST = '/content/drive/My Drive/Университет/Credit_Scoring/data/feature_extraction/test/'


##  Извлечение признаков

In [4]:
class Feature:

    def read_parquet_dataset_from_local(path_to_dataset: str, start_from: int = 0,
                                        num_parts_to_read: int = 2, columns=None, verbose=False) -> pd.DataFrame:
        """
        читает num_parts_to_read партиций, преобразует их к pd.DataFrame и возвращает
        :param path_to_dataset: путь до директории с партициями
        :param start_from: номер партиции, с которой начать чтение
        :param num_parts_to_read: количество партиций, которые требуется прочитать
        :param columns: список колонок, которые нужно прочитать из партиции
        :return: pd.DataFrame
        """

        res = []
        dataset_paths = sorted([os.path.join(path_to_dataset, filename) for filename in os.listdir(path_to_dataset)
                                  if filename.startswith('part')])  # Получаем список всех файлов, начинающихся с 'part'

        start_from = max(0, start_from)  # Убеждаемся, что стартовая партиция не меньше нуля
        chunks = dataset_paths[start_from: start_from + num_parts_to_read]  # Выбираем нужное количество партиций для чтения
        if verbose:
            print('Reading chunks:\n')  # Если verbose=True, выводим на печать имена партиций
            for chunk in chunks:
                print(chunk)
        for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):  # Читаем партиции с отображением прогресса
            chunk = pd.read_parquet(chunk_path, columns=columns)  # Читаем текущую партицию, возможно, выбирая определенные колонки
            res.append(chunk)  # Добавляем прочитанную партицию в список
        return pd.concat(res).reset_index(drop=True)  # Объединяем все партиции в один DataFrame и сбрасываем индексы

        # константны для признаков танзакции
        CAT_COLUMNS = ['currency', 'operation_kind', 'card_type',
                      'operation_type', 'operation_type_group', 'ecommerce_flag',
                      'payment_system', 'income_flag', 'mcc', 'country', 'city',
                      'mcc_category', 'day_of_week', 'hour','weekofyear']

        NUMERIC_COLUMNS = ['days_before', 'hour_diff']

        REAL_COLUMNS = ['amnt']


    def __amnt_pivot_table_by_column_as_frame(frame, column, agg_funcs=None) -> pd.DataFrame:
        """
        Строит pivot table для между колонкой `amnt`  и column на основе переданных aggregations_on
        :param frame: pd.DataFrame транзакций
        :param column: название колонки, на основе `amnt`  и column будет построен pivot_table
        :param agg_funcs: список из функций, которые нужно применить, по умолчанию ['mean', 'count']
        :return: pd.DataFrame
        """
        if agg_funcs is None:
            agg_funcs = ['mean', 'count']
        aggs = pd.pivot_table(frame, values='amnt',
                              index=['app_id'], columns=[column],
                              aggfunc={'amnt': agg_funcs},
                              fill_value=0.0)
        aggs.columns = [f'{col[0]}_{column}_{col[1]}' for col in aggs.columns.values]
        return aggs


    def extract_basic_aggregations(transactions_frame: pd.DataFrame, cat_columns=None, agg_funcs=None) -> pd.DataFrame:
        """
        :param transactions_frame: pd.DataFrame с транзакциями
        :param cat_columns: список категориальных переменных, для которых будут построены агрегаты по `amnt`
        :param agg_funcs: список функций, который нужно применить для подсчета агрегатов, по умолчанию
        ['sum', 'mean', 'count']
        :return: pd.DataFrame с извлеченными признаками
        """
        if not cat_columns:
            cat_columns = Feature.CAT_COLUMNS

        pivot_tables = []
        for col in cat_columns:
            pivot_tables.append(Feature.__amnt_pivot_table_by_column_as_frame(transactions_frame, column=col,
                                                                      agg_funcs=agg_funcs))
        pivot_tables = pd.concat(pivot_tables, axis=1)

        aggs = {
            # посчитаем статистики для транзакций
            'amnt': ['mean', 'median', 'sum', 'std'],
            # посчитаем разумные агрегаты для разницы в часах между транзакциями
            'hour_diff': ['max', 'mean', 'median', 'var', 'std'],
            # добавим самую раннюю/позднюю и среднюю дату транзакции до подачи заявки на кредит
            'days_before': ['min', 'max', 'median']}

        numeric_stats = transactions_frame.groupby(['app_id']).agg(aggs)

        # дадим разумные имена новым колонкам;
        numeric_stats.columns = [k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

        return pd.concat([pivot_tables, numeric_stats], axis=1).reset_index()


    def prepare_transactions_dataset(path_to_dataset: str, num_parts_to_preprocess_at_once: int = 1, num_parts_total: int=50,
                              save_to_path=None, verbose: bool=False):
        """
        возвращает готовый pd.DataFrame с признаками, на которых можно учить модель для целевой задачи.
        path_to_dataset: str
            путь до датасета с партициями
        num_parts_to_preprocess_at_once: int
            количество партиций, которые будут одновременно держаться в памяти и обрабатываться
        num_parts_total: int
            общее количество партиций, которые нужно обработать
        save_to_path: str
            путь до папки, в которой будет сохранен каждый обработанный блок в .parquet формате. Если None, то не будет сохранен
        verbose: bool
            логирует каждый обрабатываемый кусок данных
        """
        preprocessed_frames = []
        block = 0
        for step in tqdm.tqdm_notebook(range(0, num_parts_total, num_parts_to_preprocess_at_once),
                                      desc="Transforming transactions data"):
            transactions_frame = Feature.read_parquet_dataset_from_local(path_to_dataset, step, num_parts_to_preprocess_at_once,
                                                                verbose=verbose)
            features = Feature.extract_basic_aggregations(transactions_frame,
                                                  cat_columns=['mcc_category', 'day_of_week', 'operation_type'])
            if save_to_path:
                block_as_str = str(block)
                if len(block_as_str) == 1:
                    block_as_str = '00' + block_as_str
                else:
                    block_as_str = '0' + block_as_str
                features.to_parquet(os.path.join(save_to_path, f'processed_chunk_{block_as_str}.parquet'))

            preprocessed_frames.append(features)
        return pd.concat(preprocessed_frames)


In [5]:
# Извлечём признаки из тренировочного дата-сета
data = Feature.prepare_transactions_dataset(TRAIN_TRANSACTIONS_PATH, num_parts_to_preprocess_at_once=2, num_parts_total=50,
                                    save_to_path=FEATURE_EXTRACTION_PATH_TRAIN)


<ipython-input-4-7e1428de76dd>:108: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for step in tqdm.tqdm_notebook(range(0, num_parts_total, num_parts_to_preprocess_at_once),


Transforming transactions data:   0%|          | 0/25 [00:00<?, ?it/s]

<ipython-input-4-7e1428de76dd>:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):  # Читаем партиции с отображением прогресса


Reading dataset with pandas:   0%|          | 0/2 [00:00<?, ?it/s]

<ipython-input-4-7e1428de76dd>:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):  # Читаем партиции с отображением прогресса


...

Reading dataset with pandas:   0%|          | 0/2 [00:00<?, ?it/s]

<ipython-input-4-7e1428de76dd>:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):  # Читаем партиции с отображением прогресса


Reading dataset with pandas:   0%|          | 0/2 [00:00<?, ?it/s]

<ipython-input-4-7e1428de76dd>:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):  # Читаем партиции с отображением прогресса


Reading dataset with pandas:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
print(f'Объем в RAM всего датасета: {round(data.memory_usage(index=True).sum() / 10**9, 3)} Gb')


Объем в RAM всего датасета: 0.975 Gb


In [7]:
# Извлечём признаки из тестового дата-сета
test_data = Feature.prepare_transactions_dataset(TEST_TRANSACTIONS_PATH, num_parts_to_preprocess_at_once=3, num_parts_total=50,
                                         save_to_path=FEATURE_EXTRACTION_PATH_TEST)


<ipython-input-4-7e1428de76dd>:108: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for step in tqdm.tqdm_notebook(range(0, num_parts_total, num_parts_to_preprocess_at_once),


Transforming transactions data:   0%|          | 0/17 [00:00<?, ?it/s]

<ipython-input-4-7e1428de76dd>:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):  # Читаем партиции с отображением прогресса


Reading dataset with pandas:   0%|          | 0/3 [00:00<?, ?it/s]

<ipython-input-4-7e1428de76dd>:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):  # Читаем партиции с отображением прогресса


...

Reading dataset with pandas:   0%|          | 0/3 [00:00<?, ?it/s]

<ipython-input-4-7e1428de76dd>:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):  # Читаем партиции с отображением прогресса


Reading dataset with pandas:   0%|          | 0/3 [00:00<?, ?it/s]

<ipython-input-4-7e1428de76dd>:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):  # Читаем партиции с отображением прогресса


Reading dataset with pandas:   0%|          | 0/2 [00:00<?, ?it/s]

## Загрузка и объединение данных

In [8]:
test_target =  pd.read_csv(TEST_TARGET_PATH)
test_target.head()


,app_id,product
0,1063620,0
1,1063621,0
2,1063622,1
3,1063623,1
4,1063624,2


In [9]:
merged_test_data = test_data.merge(test_target[['app_id', 'product']], on='app_id')
merged_test_data


,app_id,count_mcc_category_1,count_mcc_category_2,count_mcc_category_3,count_mcc_category_4,count_mcc_category_5,count_mcc_category_6,count_mcc_category_7,count_mcc_category_8,count_mcc_category_9,count_mcc_category_10,count_mcc_category_11,count_mcc_category_12,count_mcc_category_13,count_mcc_category_14,count_mcc_category_15,count_mcc_category_16,count_mcc_category_17,count_mcc_category_18,count_mcc_category_19,count_mcc_category_20,count_mcc_category_21,count_mcc_category_22,count_mcc_category_23,count_mcc_category_24,count_mcc_category_25,count_mcc_category_26,count_mcc_category_27,count_mcc_category_28,mean_mcc_category_1,mean_mcc_category_2,mean_mcc_category_3,mean_mcc_category_4,mean_mcc_category_5,mean_mcc_category_6,mean_mcc_category_7,mean_mcc_category_8,mean_mcc_category_9,mean_mcc_category_10,mean_mcc_category_11,mean_mcc_category_12,mean_mcc_category_13,mean_mcc_category_14,mean_mcc_category_15,mean_mcc_category_16,mean_mcc_category_17,mean_mcc_category_18,mean_mcc_category_19,mean_mcc_category_20,mean_mcc_category_21,mean_mcc_category_22,mean_mcc_category_23,mean_mcc_category_24,mean_mcc_category_25,mean_mcc_category_26,mean_mcc_category_27,mean_mcc_category_28,count_day_of_week_1,count_day_of_week_2,count_day_of_week_3,count_day_of_week_4,count_day_of_week_5,count_day_of_week_6,count_day_of_week_7,mean_day_of_week_1,mean_day_of_week_2,mean_day_of_week_3,mean_day_of_week_4,mean_day_of_week_5,mean_day_of_week_6,mean_day_of_week_7,count_operation_type_1,count_operation_type_2,count_operation_type_3,count_operation_type_4,count_operation_type_5,count_operation_type_6,count_operation_type_7,count_operation_type_8,count_operation_type_9,count_operation_type_10,count_operation_type_11,count_operation_type_12,count_operation_type_13,count_operation_type_14,count_operation_type_15,count_operation_type_16,count_operation_type_17,count_operation_type_18,count_operation_type_19,count_operation_type_20,count_operation_type_21,count_operation_type_22,mean_operation_type_1,mean_operation_type_2,mean_operation_type_3,mean_operation_type_4,mean_operation_type_5,mean_operation_type_6,mean_operation_type_7,mean_operation_type_8,mean_operation_type_9,mean_operation_type_10,mean_operation_type_11,mean_operation_type_12,mean_operation_type_13,mean_operation_type_14,mean_operation_type_15,mean_operation_type_16,mean_operation_type_17,mean_operation_type_18,mean_operation_type_19,mean_operation_type_20,mean_operation_type_21,mean_operation_type_22,amnt_mean,amnt_median,amnt_sum,amnt_std,hour_diff_max,hour_diff_mean,hour_diff_median,hour_diff_var,hour_diff_std,days_before_min,days_before_max,days_before_median,product
0,1063620,67,87,224,79,12,72,20,14,44,1,3,3,9,0,13,9,6,1,8,0,0,0,0,0,0,1,0,0,0.374893,0.450460,0.232218,0.339087,0.374568,0.304501,0.356436,0.410680,0.413262,0.464859,0.404743,0.381515,0.441680,0.000000,0.442177,0.448111,0.421683,0.429013,0.368924,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.579884,0.0,0.0,106,87,105,87,92,87,109,0.334436,0.331190,0.335415,0.327747,0.361581,0.305618,0.328561,0,549,3,29,26,26,0,8,22,0,9,0,1,0,0,0,0,0,0,0,0,0,0.000000,0.310710,0.508202,0.529622,0.432579,0.361804,0.000000,0.459533,0.351000,0.000000,0.413964,0.000000,0.422172,0.0,0.0,0.000000,0.0,0.000000,0.000000,0,0.0,0.000000,0.332338,0.354278,223.663752,0.143467,238,12.716196,4.0,558.742252,23.637729,1,359,108.0,0
1,1063621,78,37,1,1,6,32,6,46,3,0,8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.333128,0.413989,0.349947,0.345966,0.334767,0.268163,0.358662,0.270007,0.434815,0.000000,0.263400,0.363534,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,33,35,25,39,25,45,17,0.347747,0.316946,0.349675,0.299930,0.353718,0.298355,0.332825,0,26,2,4,25,15,0,14,0,0,0,125,4,0,0,0,0,3,0,0,0,1,0.000000,0.350639,0.463622,0.534908,0.395807,0.355931,0.000000,0.469286,0.000000,0.000000,0.000000,0.278351,0.286057,0.0,0.0,0.000000,0.0,0.336356,0.000000,0,0.0,0.000000,0.323903,0.324725,70.934822,0

In [10]:
targets = pd.read_csv(TRAIN_TARGET_PATH)
targets.head()


,app_id,product,flag
0,0,3,0
1,1,1,0
2,2,1,0
3,3,1,0
4,4,1,0


In [11]:
merged_data = data.merge(targets[['app_id', 'product', 'flag']], on=['app_id'])
merged_data


,app_id,count_mcc_category_1,count_mcc_category_2,count_mcc_category_3,count_mcc_category_4,count_mcc_category_5,count_mcc_category_6,count_mcc_category_7,count_mcc_category_8,count_mcc_category_9,count_mcc_category_10,count_mcc_category_11,count_mcc_category_12,count_mcc_category_13,count_mcc_category_14,count_mcc_category_15,count_mcc_category_16,count_mcc_category_17,count_mcc_category_18,count_mcc_category_19,count_mcc_category_20,count_mcc_category_21,count_mcc_category_22,count_mcc_category_23,count_mcc_category_24,count_mcc_category_25,count_mcc_category_26,count_mcc_category_27,count_mcc_category_28,mean_mcc_category_1,mean_mcc_category_2,mean_mcc_category_3,mean_mcc_category_4,mean_mcc_category_5,mean_mcc_category_6,mean_mcc_category_7,mean_mcc_category_8,mean_mcc_category_9,mean_mcc_category_10,mean_mcc_category_11,mean_mcc_category_12,mean_mcc_category_13,mean_mcc_category_14,mean_mcc_category_15,mean_mcc_category_16,mean_mcc_category_17,mean_mcc_category_18,mean_mcc_category_19,mean_mcc_category_20,mean_mcc_category_21,mean_mcc_category_22,mean_mcc_category_23,mean_mcc_category_24,mean_mcc_category_25,mean_mcc_category_26,mean_mcc_category_27,mean_mcc_category_28,count_day_of_week_1,count_day_of_week_2,count_day_of_week_3,count_day_of_week_4,count_day_of_week_5,count_day_of_week_6,count_day_of_week_7,mean_day_of_week_1,mean_day_of_week_2,mean_day_of_week_3,mean_day_of_week_4,mean_day_of_week_5,mean_day_of_week_6,mean_day_of_week_7,count_operation_type_1,count_operation_type_2,count_operation_type_3,count_operation_type_4,count_operation_type_5,count_operation_type_6,count_operation_type_7,count_operation_type_8,count_operation_type_9,count_operation_type_10,count_operation_type_11,count_operation_type_13,count_operation_type_14,count_operation_type_15,count_operation_type_16,count_operation_type_17,count_operation_type_19,count_operation_type_20,count_operation_type_21,count_operation_type_22,mean_operation_type_1,mean_operation_type_2,mean_operation_type_3,mean_operation_type_4,mean_operation_type_5,mean_operation_type_6,mean_operation_type_7,mean_operation_type_8,mean_operation_type_9,mean_operation_type_10,mean_operation_type_11,mean_operation_type_13,mean_operation_type_14,mean_operation_type_15,mean_operation_type_16,mean_operation_type_17,mean_operation_type_19,mean_operation_type_20,mean_operation_type_21,mean_operation_type_22,amnt_mean,amnt_median,amnt_sum,amnt_std,hour_diff_max,hour_diff_mean,hour_diff_median,hour_diff_var,hour_diff_std,days_before_min,days_before_max,days_before_median,count_operation_type_12,count_operation_type_18,mean_operation_type_12,mean_operation_type_18,product,flag
0,0,38,109,1,1,0,0,11,4,6,0,4,0,1,3,0,1,0,0,0,0,2,0,0,0,0,0,0,0,0.314338,0.411590,0.282643,0.258414,0.000000,0.000000,0.358983,0.476752,0.417680,0.000000,0.336135,0.000000,0.450299,0.358530,0.000000,0.348838,0.000000,0.000000,0.000000,0.000000,0.526191,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,18,28,20,31,29,31,24,0.407102,0.367082,0.405063,0.370230,0.389952,0.397770,0.381615,5,58,70,10,14,1,12,2,1,4,0,3,0,0,1,0,0,0,0,0,0.391478,0.334183,0.423406,0.450615,0.400463,0.426544,0.325863,0.387677,0.527853,0.476752,0.000000,0.341430,0.0,0.000000,0.320248,0.0,0.0,0,0.000000,0.0,0.386645,0.387677,69.982751,0.094032,555,44.613260,20.0,5505.749601,74.200739,14,351,140.0,NaN,NaN,NaN,NaN,3,0
1,1,82,98,21,25,22,7,20,0,4,3,53,3,0,10,5,0,1,2,0,0,0,0,0,0,0,0,0,0,0.308654,0.402700,0.324518,0.290052,0.368639,0.283080,0.288406,0.000000,0.415924,0.372053,0.275840,0.325762,0.000000,0.370235,0.315308,0.000000,0.406542,0.311260,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,41,46,77,65,44,48,35,0.342776,0.340613,0.346442,0.330076,0.315696,0.322229,0.347832,0,253,83,9,0,0,0,6,0,0,0,5,0,0,0,0,0,0,0,0,0.000000,0.310619,0.400795,0.404996,0.000000,0.000000,0.000000,0.425604,0.000000,0.000000,0.000000,0.266696,0.0,0.000000,0.000000,0.0,0.0,0,0.000000,0.0,0.335351,0.336345,119.384783,0.078044,517,24.053371,14.0,2624.771791,51.232527,1,3

In [12]:
features = [x for x in merged_data.columns if x not in ['app_id', 'flag']]
features


['count_mcc_category_1',
 'count_mcc_category_2',
 'count_mcc_category_3',
 'count_mcc_category_4',
 'count_mcc_category_5',
 'count_mcc_category_6',
 'count_mcc_category_7',
 'count_mcc_category_8',
 'count_mcc_category_9',
 'count_mcc_category_10',
 'count_mcc_category_11',
 'count_mcc_category_12',
 'count_mcc_category_13',
 'count_mcc_category_14',
 'count_mcc_category_15',
 'count_mcc_category_16',
 'count_mcc_category_17',
 'count_mcc_category_18',
 'count_mcc_category_19',
 'count_mcc_category_20',
 'count_mcc_category_21',
 'count_mcc_category_22',
 'count_mcc_category_23',
 'count_mcc_category_24',
 'count_mcc_category_25',
 'count_mcc_category_26',
 'count_mcc_category_27',
 'count_mcc_category_28',
 'mean_mcc_category_1',
 'mean_mcc_category_2',
 'mean_mcc_category_3',
 'mean_mcc_category_4',
 'mean_mcc_category_5',
 'mean_mcc_category_6',
 'mean_mcc_category_7',
 'mean_mcc_category_8',
 'mean_mcc_category_9',
 'mean_mcc_category_10',
 'mean_mcc_category_11',
 'mean_mcc_cate

## Обучение модели LightGBM на CPU

In [38]:
targets = merged_data.flag.values

cv = KFold(n_splits=5, random_state=100, shuffle=True)

oof = np.zeros(len(merged_data))
train_preds = np.zeros(len(merged_data))

models = []

tree_params = {
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'max_depth': 3,
    'reg_lambda': 1,
    'num_leaves': 64,
    'n_jobs': 5,
    'n_estimators': 1_000
}

for fold_, (train_idx, val_idx) in enumerate(cv.split(merged_data, targets), 1):
    print(f'Training with fold {fold_} started.')

    train, val = merged_data.iloc[train_idx], merged_data.iloc[val_idx]

    train_data = lgb.Dataset(train[features], label=train.flag.values)
    val_data = lgb.Dataset(val[features], label=val.flag.values, reference=train_data)

    bst = lgb.train(
        params=tree_params,
        train_set=train_data,
        valid_sets=[train_data, val_data],
        valid_names=['train', 'valid'],
        callbacks=[lgb.early_stopping(stopping_rounds=2), lgb.log_evaluation(period=50)]
    )

    oof[val_idx] = bst.predict(val[features], num_iteration=bst.best_iteration)
    train_preds[train_idx] += bst.predict(train[features], num_iteration=bst.best_iteration) / (cv.n_splits - 1)
    models.append(bst)
    print(f'Training with fold {fold_} completed.')


Training with fold 1 started.


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 21300, number of negative: 749748
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.458712 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26582
[LightGBM] [Info] Number of data points in the train set: 771048, number of used features: 126
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.027625 -> initscore=-3.561030
[LightGBM] [Info] Start training from score -3.561030
Training until validation scores don't improve for 2 rounds
[50]	train's auc: 0.741709	valid's auc: 0.733569
[100]	train's auc: 0.756043	valid's auc: 0.745821
[150]	train's auc: 0.763203	valid's auc: 0.751263
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 21263, number of negative: 749786
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.915802 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 26578
[LightGBM] [Info] Number of data points in the train set: 771049, number of used features: 126
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.027577 -> initscore=-3.562819
[LightGBM] [Info] Start training from score -3.562819
Training until validation scores don't improve for 2 rounds
[50]	train's auc: 0.741541	valid's auc: 0.735806
[100]	train's auc: 0.756017	valid's auc: 0.748626
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[150]	train's auc: 0.763168	valid's auc: 0.75423
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Ligh

...

[LightGBM] [Info] Number of positive: 21268, number of negative: 749781
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.494252 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26597
[LightGBM] [Info] Number of data points in the train set: 771049, number of used features: 126
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.027583 -> initscore=-3.562578
[LightGBM] [Info] Start training from score -3.562578
Training until validation scores don't improve for 2 rounds
[50]	train's auc: 0.73978	valid's auc: 0.738215
[100]	train's auc: 0.754901	valid's auc: 0.75175
[150]	train's auc: 0.761721	valid's auc: 0.757028
[200]	train's auc: 0.766646	valid's auc: 0.760752
[250]	train's auc: 0.770507	valid's auc: 0.763305
[300]	train's auc: 0.773883	valid's auc: 0.765486
Early stopping, best iteration is:
[338]	train's auc: 0.775954	valid's auc: 0.76665
Training with fold 3 completed.
Training with fold 4 started.

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 21249, number of negative: 749800
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.254721 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26585
[LightGBM] [Info] Number of data points in the train set: 771049, number of used features: 126
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.027559 -> initscore=-3.563497
[LightGBM] [Info] Start training from score -3.563497
Training until validation scores don't improve for 2 rounds
[50]	train's auc: 0.741029	valid's auc: 0.730437
[100]	train's auc: 0.756072	valid's auc: 0.744672
[150]	train's auc: 0.763227	valid's auc: 0.75036
[200]	train's auc: 0.768243	valid's auc: 0.75434
[250]	train's auc: 0.772044	valid's auc: 0.756996
[300]	train's auc: 0.775198	valid's auc: 0.759049
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 21228, number of negative: 749821
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.437962 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26656
[LightGBM] [Info] Number of data points in the train set: 771049, number of used features: 126
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.027531 -> initscore=-3.564513
[LightGBM] [Info] Start training from score -3.564513
Training until validation scores don't improve for 2 rounds
[50]	train's auc: 0.738671	valid's auc: 0.737027
[100]	train's auc: 0.753891	valid's auc: 0.750886
[150]	train's auc: 0.761097	valid's auc: 0.756315
[200]	train's auc: 0.76644	valid's auc: 0.760125
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[250]	train's auc: 0.770219	valid's auc: 0.762406
[LightGBM] [Warning] No further splits with 

### Скоры

In [39]:
'Train roc-auc', roc_auc_score(targets, train_preds)


('Train roc-auc', 0.778674690234171)

In [40]:
'CV roc-auc', roc_auc_score(targets, oof)


('CV roc-auc', 0.7635313699625546)

## Обучение модели Catboost на GPU

In [13]:
targets = merged_data.flag.values

cv = KFold(n_splits=5, random_state=100, shuffle=True)

oof = np.zeros(len(merged_data))
train_preds = np.zeros(len(merged_data))

models = []


tree_params = {
    'max_depth': 5,
    'eval_metric': 'AUC',
    'loss_function': 'Logloss',
    'random_state': 100,
    'l2_leaf_reg': 1,
    'task_type': 'GPU' # если имеется GPU с CUDA, то можно ее задействовать, чтобы на порядок ускорить обучение
}


for fold_, (train_idx, val_idx) in enumerate(cv.split(merged_data, targets), 1):
    print(f'Training with fold {fold_} started.')
    model = cb.CatBoostClassifier(**tree_params)

    train, val = merged_data.iloc[train_idx], merged_data.iloc[val_idx]

    train_pool = cb.Pool(train[features], train.flag.values)
    val_pool = cb.Pool(val[features], val.flag.values)


    model.fit(train_pool, eval_set=[val_pool], early_stopping_rounds=100, verbose_eval=50, use_best_model=True, plot=False)

    oof[val_idx] = model.predict_proba(val_pool)[:, 1]

    train_preds[train_idx] += model.predict_proba(train_pool)[:, 1] / (cv.n_splits-1)
    models.append(model)
    print(f'Training with fold {fold_} completed.')

Training with fold 1 started.


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6316783	best: 0.6316783 (0)	total: 494ms	remaining: 8m 13s
50:	test: 0.7359870	best: 0.7359870 (50)	total: 1.49s	remaining: 27.7s
100:	test: 0.7418430	best: 0.7418430 (100)	total: 2.35s	remaining: 20.9s
150:	test: 0.7456670	best: 0.7456670 (150)	total: 3.28s	remaining: 18.4s
200:	test: 0.7489372	best: 0.7489372 (200)	total: 4.1s	remaining: 16.3s
250:	test: 0.7513135	best: 0.7513135 (250)	total: 5.01s	remaining: 15s
300:	test: 0.7527898	best: 0.7527898 (300)	total: 5.81s	remaining: 13.5s
350:	test: 0.7540968	best: 0.7540968 (350)	total: 6.59s	remaining: 12.2s
400:	test: 0.7551830	best: 0.7551838 (399)	total: 7.38s	remaining: 11s
450:	test: 0.7561705	best: 0.7561705 (450)	total: 8.14s	remaining: 9.91s
500:	test: 0.7569168	best: 0.7569168 (500)	total: 8.94s	remaining: 8.9s
550:	test: 0.7576693	best: 0.7576693 (550)	total: 10.6s	remaining: 8.63s
600:	test: 0.7582715	best: 0.7582733 (598)	total: 13.2s	remaining: 8.78s
650:	test: 0.7588412	best: 0.7588437 (649)	total: 14.3s	remain

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6273480	best: 0.6273480 (0)	total: 30.8ms	remaining: 30.8s
50:	test: 0.7359306	best: 0.7359306 (50)	total: 1.01s	remaining: 18.8s
100:	test: 0.7432002	best: 0.7432002 (100)	total: 1.88s	remaining: 16.7s
150:	test: 0.7476014	best: 0.7476014 (150)	total: 2.72s	remaining: 15.3s
200:	test: 0.7508373	best: 0.7508373 (200)	total: 3.52s	remaining: 14s
250:	test: 0.7531317	best: 0.7531317 (250)	total: 4.33s	remaining: 12.9s
300:	test: 0.7547815	best: 0.7547815 (300)	total: 5.21s	remaining: 12.1s
350:	test: 0.7561931	best: 0.7561931 (350)	total: 6s	remaining: 11.1s
400:	test: 0.7574802	best: 0.7574802 (400)	total: 6.78s	remaining: 10.1s
450:	test: 0.7584585	best: 0.7584585 (450)	total: 7.56s	remaining: 9.21s
500:	test: 0.7592531	best: 0.7592531 (500)	total: 8.35s	remaining: 8.31s
550:	test: 0.7598011	best: 0.7598019 (546)	total: 9.36s	remaining: 7.63s
600:	test: 0.7604136	best: 0.7604136 (600)	total: 11.7s	remaining: 7.76s
650:	test: 0.7610429	best: 0.7610429 (650)	total: 13.4s	remai

...

0:	test: 0.6359841	best: 0.6359841 (0)	total: 31.2ms	remaining: 31.2s
50:	test: 0.7397622	best: 0.7397622 (50)	total: 1s	remaining: 18.7s
100:	test: 0.7455820	best: 0.7455820 (100)	total: 1.89s	remaining: 16.8s
150:	test: 0.7497079	best: 0.7497079 (150)	total: 2.72s	remaining: 15.3s
200:	test: 0.7527815	best: 0.7527815 (200)	total: 3.57s	remaining: 14.2s
250:	test: 0.7555961	best: 0.7555961 (250)	total: 4.47s	remaining: 13.3s
300:	test: 0.7571081	best: 0.7571081 (300)	total: 6.54s	remaining: 15.2s
350:	test: 0.7585376	best: 0.7585376 (350)	total: 9.13s	remaining: 16.9s
400:	test: 0.7598209	best: 0.7598209 (400)	total: 10.6s	remaining: 15.9s
450:	test: 0.7607880	best: 0.7607880 (450)	total: 13s	remaining: 15.8s
500:	test: 0.7617958	best: 0.7617958 (500)	total: 14.6s	remaining: 14.5s
550:	test: 0.7625109	best: 0.7625109 (550)	total: 15.3s	remaining: 12.5s
600:	test: 0.7632868	best: 0.7632870 (599)	total: 16.1s	remaining: 10.7s
650:	test: 0.7640816	best: 0.7640816 (650)	total: 16.9s	remai

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6322695	best: 0.6322695 (0)	total: 31.2ms	remaining: 31.2s
50:	test: 0.7324776	best: 0.7325769 (49)	total: 999ms	remaining: 18.6s
100:	test: 0.7393034	best: 0.7393034 (100)	total: 1.88s	remaining: 16.7s
150:	test: 0.7441899	best: 0.7441899 (150)	total: 2.73s	remaining: 15.3s
200:	test: 0.7478303	best: 0.7478303 (200)	total: 3.59s	remaining: 14.3s
250:	test: 0.7500377	best: 0.7500377 (250)	total: 4.41s	remaining: 13.2s
300:	test: 0.7517632	best: 0.7517632 (300)	total: 5.24s	remaining: 12.2s
350:	test: 0.7528809	best: 0.7528809 (350)	total: 6.03s	remaining: 11.2s
400:	test: 0.7540504	best: 0.7540504 (400)	total: 7.38s	remaining: 11s
450:	test: 0.7550642	best: 0.7550642 (450)	total: 9.52s	remaining: 11.6s
500:	test: 0.7560252	best: 0.7560252 (500)	total: 11.5s	remaining: 11.5s
550:	test: 0.7567511	best: 0.7567511 (550)	total: 12.3s	remaining: 10s
600:	test: 0.7574911	best: 0.7574911 (600)	total: 13.1s	remaining: 8.67s
650:	test: 0.7579889	best: 0.7579889 (650)	total: 13.9s	rema

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6386386	best: 0.6386386 (0)	total: 32ms	remaining: 31.9s
50:	test: 0.7407731	best: 0.7407731 (50)	total: 1.01s	remaining: 18.8s
100:	test: 0.7469451	best: 0.7469451 (100)	total: 1.89s	remaining: 16.8s
150:	test: 0.7506180	best: 0.7506180 (150)	total: 2.75s	remaining: 15.5s
200:	test: 0.7539460	best: 0.7539460 (200)	total: 3.58s	remaining: 14.2s
250:	test: 0.7562898	best: 0.7562898 (250)	total: 4.4s	remaining: 13.1s
300:	test: 0.7581371	best: 0.7581371 (300)	total: 5.2s	remaining: 12.1s
350:	test: 0.7596453	best: 0.7596456 (349)	total: 6.06s	remaining: 11.2s
400:	test: 0.7606209	best: 0.7606209 (400)	total: 6.9s	remaining: 10.3s
450:	test: 0.7616511	best: 0.7616511 (450)	total: 8.11s	remaining: 9.87s
500:	test: 0.7623832	best: 0.7623832 (500)	total: 10.5s	remaining: 10.4s
550:	test: 0.7629711	best: 0.7629711 (550)	total: 12.3s	remaining: 10s
600:	test: 0.7635171	best: 0.7635171 (600)	total: 13.1s	remaining: 8.69s
650:	test: 0.7640242	best: 0.7640242 (650)	total: 13.9s	remaini

### Скоры

In [14]:
'Train roc-auc', roc_auc_score(targets, train_preds)


('Train roc-auc', 0.7758897350547598)

In [15]:
'CV roc-auc', roc_auc_score(targets, oof)


('CV roc-auc', 0.7641613449172037)